# Benchmark Manifold GP Semi-Supervised Learning

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
import os
import scipy.spatial as ss
from time import time
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from gpytorch.priors import NormalPrior, GammaPrior

from manifold_gp.utils.torch_helper import memory_allocation

## Dataset Preprocessing

### Load & Settings

In [2]:
torch.manual_seed(1337)
dataset = 'mnist'

data = np.loadtxt('datasets/'+dataset+'_train.csv')
sampled_x, sampled_y = data[:, 2:], data[:, 1]
rand_idx = torch.randperm(sampled_x.shape[0])
sampled_x, sampled_y = sampled_x[rand_idx], sampled_y[rand_idx]
del rand_idx

data = np.loadtxt('datasets/'+dataset+'_test.csv')
test_x, test_y = data[:, 2:], data[:, 1]
    
preprocess = False
normalize_features = False
normalize_labels = True

In [3]:
if preprocess:
    # remove coincident points
    sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    sampled_y = sampled_y[id_unique]

    # cut between 0.1 and 0.9 percentile of distances
    import faiss
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(sampled_x)
    knn.add(sampled_x)
    v = np.sqrt(knn.search(sampled_x, 51)[0][:,1:])
    idx = np.argsort(v.mean(axis=1).ravel())
    percentile_start = int(np.round(idx.shape[0]*0.10))
    percentile_end = int(np.round(idx.shape[0]*0.90))
    sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
    sampled_y = sampled_y[idx[percentile_start:percentile_end]]
    del knn
m = sampled_x.shape[0]

### Trainset & Testset

In [4]:
split = int(0.1 * m)
train_idx = torch.arange(0, split)
train_x, train_y = sampled_x[:split], sampled_y[:split]

sampled_x = torch.from_numpy(sampled_x).float()
train_x, train_y = torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()
test_x, test_y = torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = sampled_x.mean(dim=-2, keepdim=True), sampled_x.std(dim=-2, keepdim=True) + 1e-6
    sampled_x.sub_(mu_x).div_(std_x)
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
sampled_x = sampled_x.contiguous()
train_idx = train_idx.contiguous()
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
sampled_x = sampled_x.to(device)
train_idx = train_idx.to(device)
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=2,
        nodes=sampled_x,
        neighbors=50,
        operator="randomwalk",
        modes=100,
        ball_scale=10.0,
        prior_bandwidth=False,
    ),
    outputscale_prior=None # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

## Train

In [7]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.epsilon': 0.5,
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

In [8]:
t0 = time()
model.manifold_informed_train(lr=1e-2, iter=100,
                              decay_step_size=100, decay_magnitude=1.0, 
                              norm_step_size=10, norm_rand_vec=100, 
                              verbose=True, save=False)
t1 = time()
print("Time: %.2g sec" % (t1 - t0))

Iter: 0, LR: 0.010, Loss: 405.043, NoiseVar: 0.010, SignalVar: 2.073, Lengthscale: 1.000, Epsilon: 0.500
Iter: 1, LR: 0.010, Loss: 400.178, NoiseVar: 0.010, SignalVar: 2.064, Lengthscale: 1.006, Epsilon: 0.496
Iter: 2, LR: 0.010, Loss: 399.025, NoiseVar: 0.010, SignalVar: 2.055, Lengthscale: 1.013, Epsilon: 0.492
Iter: 3, LR: 0.010, Loss: 392.599, NoiseVar: 0.010, SignalVar: 2.046, Lengthscale: 1.019, Epsilon: 0.488
Iter: 4, LR: 0.010, Loss: 386.529, NoiseVar: 0.010, SignalVar: 2.038, Lengthscale: 1.025, Epsilon: 0.484
Iter: 5, LR: 0.010, Loss: 382.694, NoiseVar: 0.010, SignalVar: 2.029, Lengthscale: 1.032, Epsilon: 0.481
Iter: 6, LR: 0.010, Loss: 378.320, NoiseVar: 0.009, SignalVar: 2.020, Lengthscale: 1.038, Epsilon: 0.477
Iter: 7, LR: 0.010, Loss: 372.158, NoiseVar: 0.009, SignalVar: 2.012, Lengthscale: 1.045, Epsilon: 0.473
Iter: 8, LR: 0.010, Loss: 366.392, NoiseVar: 0.009, SignalVar: 2.003, Lengthscale: 1.051, Epsilon: 0.469
Iter: 9, LR: 0.010, Loss: 360.961, NoiseVar: 0.009, Sig

Iter: 78, LR: 0.010, Loss: -515.088, NoiseVar: 0.004, SignalVar: 1.635, Lengthscale: 1.706, Epsilon: 0.233
Iter: 79, LR: 0.010, Loss: -552.300, NoiseVar: 0.004, SignalVar: 1.635, Lengthscale: 1.716, Epsilon: 0.230
Iter: 80, LR: 0.010, Loss: -536.921, NoiseVar: 0.004, SignalVar: 1.635, Lengthscale: 1.726, Epsilon: 0.228
Iter: 81, LR: 0.010, Loss: -543.395, NoiseVar: 0.003, SignalVar: 1.637, Lengthscale: 1.736, Epsilon: 0.226
Iter: 82, LR: 0.010, Loss: -564.579, NoiseVar: 0.003, SignalVar: 1.640, Lengthscale: 1.746, Epsilon: 0.223
Iter: 83, LR: 0.010, Loss: -564.736, NoiseVar: 0.003, SignalVar: 1.645, Lengthscale: 1.756, Epsilon: 0.221
Iter: 84, LR: 0.010, Loss: -562.913, NoiseVar: 0.003, SignalVar: 1.651, Lengthscale: 1.766, Epsilon: 0.219
Iter: 85, LR: 0.010, Loss: -565.581, NoiseVar: 0.003, SignalVar: 1.659, Lengthscale: 1.776, Epsilon: 0.217
Iter: 86, LR: 0.010, Loss: -562.583, NoiseVar: 0.003, SignalVar: 1.668, Lengthscale: 1.785, Epsilon: 0.215
Iter: 87, LR: 0.010, Loss: -550.045, 

## Evaluation

In [12]:
%%capture
likelihood.eval()
model.eval()


## Metrics

In [13]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_test = likelihood(model(test_x))

    mean_test = preds_test.mean
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    model._clear_cache()
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.0183, device='cuda:0')
NLL:  tensor(-1.9455, device='cuda:0')
